In [2]:
import numpy as np
import pandas as pd

In [3]:
#data1 = pd.read_csv("../data/AAPL_05222012_0930_1300_LOB_2.csv")

In [4]:
data1 = pd.read_csv("AAPL_05222012_0930_1300_LOB_2.csv")

In [6]:
# column name
col1 = list(data1.columns.values)[2:62]
len(col1)
col2 = [i[16:] for i in col1]
new_names = list(data1.columns.values)[0:2]+ [i[:-19] for i in col2]
data1.columns = new_names

In [7]:
# delete times
data1 = data1.head(203350)
data1.shape

(203350, 62)

In [8]:
data1.head(10)

,Index,Time,BID_PRICE1,BID_UPDATE_TIME1,BID_SIZE1,ASK_PRICE1,ASK_UPDATE_TIME1,ASK_SIZE1,BID_PRICE2,BID_UPDATE_TIME2,...,BID_SIZE9,ASK_PRICE9,ASK_UPDATE_TIME9,ASK_SIZE9,BID_PRICE10,BID_UPDATE_TIME10,BID_SIZE10,ASK_PRICE10,ASK_UPDATE_TIME10,ASK_SIZE10
0,1,2012/05/22 09:30:00.000,569.02,2012/05/22 09:29:34.006,40,570.0,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200.0,571.00,2012/05/22 09:29:49.424,120.0,567.16,2012/05/22 09:28:28.730,60.0,571.24,2012/05/22 08:13:36.797,200.0
1,2,2012/05/22 09:30:00.003,569.02,2012/05/22 09:29:34.006,40,570.0,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200.0,570.99,2012/05/22 09:29:53.347,200.0,567.16,2012/05/22 09:28:28.730,60.0,571.00,2012/05/22 09:29:49.424,120.0
2,3,2012/05/22 09:30:00.003,569.02,2012/05/22 09:29:34.006,40,570.0,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200.0,570.67,2012/05/22 09:29:29.034,10.0,567.16,2012/05/22 09:28:28.730,60.0,570.99,2012/05/22 09:29:53.347,200.0
3,4,2012/05/22 09:30:00.003,569.03,2012/05/22 09:30:00.003,8,570.0,2012/05/22 09:29:43.573,400,569.02,2012/05/22 09:29:34.006,...,200.0,570.67,2012/05/22 09:29:29.034,10.0,567.30,2012/05/22 09:29:38.985,200.0,570.99,2012/05/22 09:29:53.347,200.0
4,5,2012/05/22 09:30:00.003,569.03,2012/05/22 09:30:00.003,8,570.0,2012/05/22 09:29:43.573,400,569.02,2012/05/22 09:29:34.006,...,200.0,570.55,2012/05/22 09:30:00.003,8.0,567.30,2012/05/22 09:29:38.985,200.0,570.67,2012/05/22 09:29:29.034,10.0
5,6,2012/05/22 09:30:00.016,569.06,2012/05/22 09:30:00.016,8,570.0,2012/05/22 09:29:43.573,400,569.03,2012/05/22 09:30:00.003,...,100.0,570.55,2012/05/22 09:30:00.003,8.0,567.35,2012/05/22 09:29:12.010,200.0,570.67,2012/05/22 09:29:29.034,10.0
6,7,2012/05/22 09:30:00.016,569.06,2012/05/22 09:30:00.016,8,570.0,2012/05/22 09:29:43.573,400,569.05,2012/05/22 09:30:00.016,...,105.0,570.55,2012/05/22 09:30:00.003,8.0,567.87,2012/05/22 09:29:52.495,100.0,570.67,2012/05/22 09:29:29.034,10.0
7,8,2012/05/22 09:30:00.050,569.06,2012/05/22 09:30:00.016,8,570.0,2012/05/22 09:30:00.050,470,569.05,2012/05/22 09:30:00.016,...,105.0,570.55,2012/05/22 09:30:00.003,8.0,567.87,2012/05/22 09:29:52.495,100.0,570.67,2012/05/22 09:29:29.034,10.0
8,9,2012/05/22 09:30:00.052,569.06,2012/05/22 09:30:00.016,8,570.0,2012/05/22 09:30:00.050,470,569.05,2012/05/22 09:30:00.016,...,105.0,570.54,2012/05/22 09:30:00.003,8.0,567.87,2012/05/22 09:29:52.495,100.0,570.55,2012/05/22 09:30:00.003,8.0
9,10,2012/05/22 09:30:00.127,569.06,2012/05/22 09:30:00.016,8,569.8,2012/05/22 09:30:00.127,50,569.05,2012/05/22 09:30:00.016,...,105.0,570.52,2012/05/22 09:30:00.003,8.0,567.87,2012/05/22 09:29:52.495,100.0,570.54,2012/05/22 09:30:00.003,8.0


In [10]:
#create desired names 
level  = range(1,11)
"BID_PRICE"+str(level[1])
bidpc_name = ["BID_PRICE"+str(level[i]) for i in range(10)]
askpc_name = ["ASK_PRICE"+str(level[i]) for i in range(10)]
bidsz_name = ["BID_SIZE"+str(level[i]) for i in range(10)]
asksz_name = ["ASK_SIZE"+str(level[i]) for i in range(10)]

In [45]:
def feature4 (data,level) :
    askpc_mean=np.sum(data1.ix[:,askpc_name],axis=1)/level
    bidpc_mean=np.sum(data1.ix[:,bidpc_name],axis=1)/level
    asksz_mean=np.sum(data1.ix[:,asksz_name],axis=1)/level
    bidsz_mean=np.sum(data1.ix[:,bidsz_name],axis=1)/level
    askpc_mean=askpc_mean.tolist()
    bidpc_mean=bidpc_mean.tolist()
    asksz_mean=asksz_mean.tolist()
    bidsz_mean=bidsz_mean.tolist() 
    v4 = pd.DataFrame({"Mean Ask Price":askpc_mean,"Mean Bid Price":bidpc_mean,
                  "Mean Ask Size":asksz_mean,"Mean Bid Size":bidsz_mean})
    return v4
v4 = feature4(data1,10)
v4.head(10)

,Mean Ask Price,Mean Ask Size,Mean Bid Price,Mean Bid Size
0,570.497,130.0,568.092,100.6
1,570.428,110.8,568.092,100.6
2,570.382,99.6,568.092,100.6
3,570.382,99.6,568.279,95.4
4,570.335,80.4,568.279,95.4
5,570.335,80.4,568.455,76.2
6,570.335,80.4,568.625,57.0
7,570.335,87.4,568.625,57.0
8,570.276,96.4,568.625,57.0
9,570.201,100.6,568.625,57.0


In [46]:
def feature5 (data):
    pc_diff = np.sum(data1.ix[:,askpc_name],axis=1)- np.sum(data1.ix[:,bidpc_name],axis=1)
    sz_diff = np.sum(data1.ix[:,asksz_name],axis=1)- np.sum(data1.ix[:,bidsz_name],axis=1)
    pc_diff = pc_diff.tolist()
    sz_diff = sz_diff.tolist()
    v5 = pd.DataFrame({"Acc. Price Diff":pc_diff,"Acc. Size Diff":sz_diff})
    return v5;
v5 = feature5(data1)
v5.head(10)

,Acc. Price Diff,Acc. Size Diff
0,24.05,294.0
1,23.36,102.0
2,22.90,-10.0
3,21.03,42.0
4,20.56,-150.0
5,18.80,42.0
6,17.10,234.0
7,17.10,304.0
8,16.51,394.0
9,15.76,436.0


In [54]:
def volumn_ratio (data):
    v_ratio=np.sum(data1.ix[:,asksz_name[0:4]],axis=1)/np.sum(data1.ix[:,bidsz_name[0:4]],axis=1)
    v_ratio=v_ratio.tolist()
    volumn_ratio = pd.DataFrame({"top five volumn ratio":v_ratio})
    return volumn_ratio;
volumn_ratio(data1).head(10)

,top five volumn ratio
0,1.676471
1,1.676471
2,1.676471
3,2.298387
4,2.298387
5,3.653846
6,8.906250
7,10.000000
8,10.781250
9,11.250000


In [ ]:
bidlist = range(2,62,6)
asklist = range(5,65,6)
def v3feature(data):
    bid = data.ix[:,bidlist]
    bid_feature = abs(bid.diff(axis = 1).ix[:,range(1,10)])
    names = ["bid"+str(i+1)+"-"+"bid"+str(i) for i in range(1,10)]
    bid_feature.columns = names
    bid_feature2 = pd.DataFrame(bid["BID_PRICE1"] - bid["BID_PRICE10"])
    bid_feature2.columns = ["bid1-bid10"]
    bid_feature = pd.concat([bid_feature,bid_feature2],axis=1)
    ask = data.ix[:,asklist]
    ask_feature = abs(ask.diff(axis = 1).ix[:,range(1,10)])
    names = ["ask"+str(i+1)+"-"+"ask"+str(i) for i in range(1,10)]
    ask_feature.columns = names
    ask_feature2 = pd.DataFrame(ask["ASK_PRICE1"] - ask["ASK_PRICE10"])
    ask_feature2.columns = ["ask1-ask10"]
    ask_feature = pd.concat([ask_feature,ask_feature2],axis=1)
    v3 = pd.concat([ask_feature,bid_feature],axis=1)
    return v3;

In [ ]:
v3 = v3feature(dat)
v3.shape

In [ ]:
volbidlist = range(4,64,6)
volasklist = range(7,67,6)
def v6feature(data,delta = 10):
    v6_1 = pd.concat([data.ix[:,bidlist],data.ix[:,asklist]],axis=1).diff(periods = delta, axis = 0).ix[delta:,:]/delta
    v6_2 = pd.concat([data.ix[:,volbidlist],data.ix[:,volasklist]],axis=1).diff(periods = delta, axis = 0).ix[delta:,:]/delta
    v6 = pd.concat([v6_1,v6_2],axis = 1)
    return v6;
v6= v6feature(dat)
v6